In [1]:
# Import Libraries
import requests
import numpy as np
import pandas as pd
from datetime import datetime
from dateutil.relativedelta import relativedelta

In [2]:
# The List of 20 type of cryptocurrency
# coin_list = ['BTC', 'ETH', 'XRP', 'ADA', 'USDT', 'DOGE', 'XLM', 'DOT', 'UNI', 'LINK', 'USDC', 'BCH', 'LTC', 'GRT', 'ETC', 'FIL', 'AAVE', 'ALGO', 'EOS']

coin_list = ['AAVE', 'ADA', 'BTC', 'DOGE', 'DOT', 'EOS', 'ETH', 'LINK', 'LTC', 'UNI', 'USDC', 'USDT', 'XLM', 'XRP']

In [3]:
#defining the dataframe
main_df = pd.DataFrame()

In [6]:
datetime(2023, 7, 29, 0, 0) - relativedelta(hours = 12)

datetime.datetime(2023, 7, 28, 12, 0)

In [17]:
for coin in ['BTC']:
    coin_df = pd.DataFrame()
    df = pd.DataFrame(index=[0])
    
    # Defining the Start Date and End Date
    datetime_end = datetime(2023, 7, 29, 0, 0)
    datetime_check = datetime(2023, 7, 1, 0, 0)
    
    while len(df) > 0:
        if datetime_end == datetime_check:
            break
        
        datetime_start = datetime_end - relativedelta(hours = 12)
        
        #Api for the scrapping
        url = 'https://production.api.coindesk.com/v2/price/values/'+ \
                coin + \
                '?start_date=' + datetime_start.strftime("%Y-%m-%dT%H:%M") + \
                '&end_date=' + datetime_end.strftime("%Y-%m-%dT%H:%M") + \
                '&ohlc=true'
        
        #we are using the request to fetch the data from the api in the json format and then storing it into the dataframe.
        temp_data = requests.get(url).json()
        df = pd.DataFrame(temp_data['data']['entries'])
        df.columns = ['Timestamp', 'Open', 'High', 'Low', 'Close']
        
        break
        
        # To handle the Missing Data
        insert_ids_list = [np.nan]
        
        '''In numerous occasions, I observed that CoinDesk did not collect every minute of data. 
        According to observation, in a normal circumstance, the timestamp would have a discrepancy
        value of 60000 for 1 minute. As a result, I can quickly tell that the missing period is the
        time gap if the row difference for the timestamp is larger than 60000. To deal with it, a
        hot-deck imputation procedure is used. To put it another way, the nearest minute data will 
        be substituted for the missing one.
        '''
        
#         while len(insert_ids_list) > 0:
#             timestamp_checking = np.array(df['Timestamp'][1:]) - np.array(df['Timestamp'][:-1])
#             insert_ids_list = np.where(timestamp_checking != 60000)[0]
#             if len(insert_ids_list) > 0:
#                 print(str(len(insert_ids_list)) + ' mismatched.')
#                 insert_ids = insert_ids_list[0]
#                 temp_df = df.iloc[insert_ids.repeat(int(timestamp_checking[insert_ids]/60000)-1)].reset_index(drop=True)
#                 temp_df['Timestamp'] = [temp_df['Timestamp'][0] + i*60000 for i in range(1, len(temp_df)+1)]
# #                 df = df.loc[:insert_ids].append(temp_df).append(df.loc[insert_ids+1:]).reset_index(drop=True)
#                 df = pd.concat(pd.concat((df.loc[:insert_ids],temp_df)), df.loc[insert_ids+1:]).reset_index(drop=True)
#                 insert_ids_list = insert_ids_list[1:]
                
        
        #adding datetime and symbol to dataframe
        df = df.drop(['Timestamp'], axis=1)
        df['Datetime'] = [datetime_end - relativedelta(minutes=len(df)-i) for i in range(0, len(df))]
#         coin_df = df.append(coin_df)
        coin_df = pd.concat((df,coin_df))
        datetime_end = datetime_start
        
    coin_df['Symbol'] = coin
#     main_df = main_df.append(coin_df)
    main_df = pd.concat((main_df, coin_df))

In [18]:
main_df = main_df[['Datetime', 'Symbol', 'Open', 'High', 'Low', 'Close']].reset_index(drop=True)
main_df

,Datetime,Symbol,Open,High,Low,Close
0,2023-07-01 00:00:00,BTC,30474.138070,30477.817151,30473.295184,30477.817151
1,2023-07-01 00:01:00,BTC,30477.701288,30477.701288,30441.645957,30441.645957
2,2023-07-01 00:02:00,BTC,30452.196541,30475.174213,30452.178017,30466.407718
3,2023-07-01 00:03:00,BTC,30465.162841,30479.440929,30458.232972,30477.787368
4,2023-07-01 00:04:00,BTC,30478.712659,30486.953887,30472.263397,30486.943840
...,...,...,...,...,...,...
40336,2023-07-28 23:55:00,BTC,29313.037858,29313.362552,29312.688128,29313.041388
40337,2023-07-28 23:56:00,BTC,29313.212798,29313.212798,29311.915858,29312.368743
40338,2023-07-28 23:57:00,BTC,29312.427534,29313.335959,29310.851762,29312.638660
40339,2023-07-28 23:58:00,BTC,29312.578070,29313.386101,29311.934400,29312.016455


In [ ]:
main_df.to_csv('main_df.csv', index=False)

In [10]:
import glob 
glob.glob('D:\Cryptocurrency Project\Data\coin*.csv')

['D:\\Cryptocurrency Project\\Data\\coin_Aave.csv',
 'D:\\Cryptocurrency Project\\Data\\coin_BinanceCoin.csv',
 'D:\\Cryptocurrency Project\\Data\\coin_Bitcoin.csv',
 'D:\\Cryptocurrency Project\\Data\\coin_Cardano.csv',
 'D:\\Cryptocurrency Project\\Data\\coin_ChainLink.csv',
 'D:\\Cryptocurrency Project\\Data\\coin_Cosmos.csv',
 'D:\\Cryptocurrency Project\\Data\\coin_CryptocomCoin.csv',
 'D:\\Cryptocurrency Project\\Data\\coin_Dogecoin.csv',
 'D:\\Cryptocurrency Project\\Data\\coin_EOS.csv',
 'D:\\Cryptocurrency Project\\Data\\coin_Ethereum.csv',
 'D:\\Cryptocurrency Project\\Data\\coin_Iota.csv',
 'D:\\Cryptocurrency Project\\Data\\coin_Litecoin.csv',
 'D:\\Cryptocurrency Project\\Data\\coin_Monero.csv',
 'D:\\Cryptocurrency Project\\Data\\coin_NEM.csv',
 'D:\\Cryptocurrency Project\\Data\\coin_Polkadot.csv',
 'D:\\Cryptocurrency Project\\Data\\coin_Solana.csv',
 'D:\\Cryptocurrency Project\\Data\\coin_Stellar.csv',
 'D:\\Cryptocurrency Project\\Data\\coin_Tether.csv',
 'D:\\Crypto

In [23]:
a = ['BTC', 'ETH', 'XRP', 'ADA', 'USDT', 'DOGE', 'XLM', 'DOT', 'UNI', 'LINK', 'USDC', 'BCH', 'LTC', 'GRT', 'ETC', 'FIL', 'AAVE', 'ALGO', 'EOS']
a.sort()
print(a)

['AAVE', 'ADA', 'ALGO', 'BCH', 'BTC', 'DOGE', 'DOT', 'EOS', 'ETC', 'ETH', 'FIL', 'GRT', 'LINK', 'LTC', 'UNI', 'USDC', 'USDT', 'XLM', 'XRP']


In [16]:
data = pd.read_csv('D:\\Cryptocurrency Project\\Data\\final.csv', parse_dates=['Date'])
data.head()

,Sr no.,Name,Symbol,Date,High,Low,Open,Close,Volume,Marketcap
0,0,Aave,AAVE,2020-10-05,55.112358,49.787900,52.675035,53.219243,0.000000e+00,8.912813e+07
1,1,Aave,AAVE,2020-10-06,53.402270,40.734578,53.291969,42.401599,5.830915e+05,7.101144e+07
2,2,Aave,AAVE,2020-10-07,42.408314,35.970690,42.399947,40.083976,6.828342e+05,6.713004e+07
3,3,Aave,AAVE,2020-10-08,44.902511,36.696057,39.885262,43.764463,1.658817e+06,2.202651e+08
4,4,Aave,AAVE,2020-10-09,47.569533,43.291776,43.764463,46.817744,8.155377e+05,2.356322e+08


In [12]:
data.shape

(37082, 10)

In [26]:
b = data.Symbol.unique()
b.sort()
print(b)

['AAVE' 'ADA' 'ATOM' 'BNB' 'BTC' 'CRO' 'DOGE' 'DOT' 'EOS' 'ETH' 'LINK'
 'LTC' 'MIOTA' 'SOL' 'TRX' 'UNI' 'USDC' 'USDT' 'WBTC' 'XEM' 'XLM' 'XMR'
 'XRP']


In [30]:
Coins = []
for elem in b:
    if elem in a:
        Coins.append(elem)
print(Coins)

['AAVE', 'ADA', 'BTC', 'DOGE', 'DOT', 'EOS', 'ETH', 'LINK', 'LTC', 'UNI', 'USDC', 'USDT', 'XLM', 'XRP']


In [43]:
new_data = data[data['Symbol'].isin(Coins)]
new_data.head()

,Sr no.,Name,Symbol,Date,High,Low,Open,Close,Volume,Marketcap
0,0,Aave,AAVE,2020-10-05,55.112358,49.787900,52.675035,53.219243,0.000000e+00,8.912813e+07
1,1,Aave,AAVE,2020-10-06,53.402270,40.734578,53.291969,42.401599,5.830915e+05,7.101144e+07
2,2,Aave,AAVE,2020-10-07,42.408314,35.970690,42.399947,40.083976,6.828342e+05,6.713004e+07
3,3,Aave,AAVE,2020-10-08,44.902511,36.696057,39.885262,43.764463,1.658817e+06,2.202651e+08
4,4,Aave,AAVE,2020-10-09,47.569533,43.291776,43.764463,46.817744,8.155377e+05,2.356322e+08


In [44]:
new_data.shape

(24754, 10)

In [46]:
new_data.to_csv('D:\\Cryptocurrency Project\\Data\\final_new.csv', index=False)

In [ ]:
import pandas as pd
import talib

# Load your data into a pandas DataFrame
data = pd.read_csv('your_data.csv')

# Calculate moving averages
data['MA10'] = talib.SMA(data['Close'], timeperiod=10)
data['MA50'] = talib.SMA(data['Close'], timeperiod=50)

# Calculate Relative Strength Index (RSI)
data['RSI'] = talib.RSI(data['Close'], timeperiod=14)

# Calculate Bollinger Bands
data['upper_band'], data['middle_band'], data['lower_band'] = talib.BBANDS(data['Close'], timeperiod=20)

# Calculate Moving Average Convergence Divergence (MACD)
macd, signal, _ = talib.MACD(data['Close'], fastperiod=12, slowperiod=26, signalperiod=9)
data['MACD'] = macd
data['MACD_Signal'] = signal

# Calculate Average True Range (ATR)
data['ATR'] = talib.ATR(data['High'], data['Low'], data['Close'], timeperiod=14)

# Calculate On-Balance Volume (OBV)
data['OBV'] = talib.OBV(data['Close'], data['Volume'])

# You can continue calculating other indicators based on your analysis requirements

# Save the updated DataFrame with calculated indicators
data.to_csv('data_with_indicators.csv', index=False)
